In [1]:
import pandas as pd
import geopandas as gpd
import os
import re
import numpy as np

In [2]:
%run ./00_subsetting_funcs.ipynb

In [3]:
urba_gov=pd.read_excel("D:/urba_gov.xlsx")

# Load input data

In [4]:
# Sales aggregated to unique customer ids
sales=pd.read_csv("C:/Users/Salif SAWADOGO/dynamic segmentation/data_acquisition/Sales data/sales_cust_level.csv")
sales=sales.rename(columns={"AMOUNT_EUR": "NSR", "AMOUNT_GROSS_EUR":"GSR"})
# Raw CMD 
cmd=pd.read_excel("D:/data_quality/data/BDD G 26.04.2021.XLSX")
cmd= cmd.rename(columns={'Client': 'CUSTOMER_COD'})

#read government urbanicity
urba_gov=pd.read_excel("D:/urba_gov.xlsx")

# Complete trade area set - prior to aggregation for neighbours
ta_og_tizi_ouzou = gpd.read_file(os.path.join('C:/Users/Salif SAWADOGO/dynamic segmentation/urbanicty/code/', 'cmd_ta_preagg.json'))
ta_og_tizi_ouzou= ta_og_tizi_ouzou.rename(columns={'Client': 'CUSTOMER_COD'})
 
# Trade area set post aggregation and formatting
ta_p_tizi_ouzou = pd.read_excel('C:/Users/Salif SAWADOGOL/dynamic segmentation/urbanicty/ta_tizi_ouzou_250.xlsx')
ta_p_tizi_ouzou= ta_p_tizi_ouzou.rename(columns={'Client': 'CUSTOMER_COD'})



# Form a combo set

1) Merge cmd (raw) and sales (aggregated to unique cust ids)

In [31]:
internal_combi = merge_custom_sets(cmd, sales)
internal_combi.shape

(64560, 87)


2) Form marrakech combo set from internal and trade area data


In [32]:
tizi_ouzou_combi = \
  combine_tizi_ouzou(
    ta_og=ta_og_tizi_ouzou, 
    ta_p=ta_p_tizi_ouzou, 
    internal_combi=internal_combi, 
    urba_gov=urba_gov,
    top_field='NSR', 
    combi_keepers=['CUSTOMER_COD', 'NSR', 'GSR', 'UNIT_CASES','TRANSACTIONS']
  )

In [38]:
channel_scope=tizi_ouzou_combi.loc[~tizi_ouzou_combi["Classification client"].isin(["Platinum","Prestigieux"])]
pd.crosstab(channel_scope["Classification client"], channel_scope.CHANNEL_CUSTOM, margins=True)

CHANNEL_CUSTOM,AG,HORECA,OTHER,SNACK,All
Classification client,,,,,
Bronze,651,998,1,92,1742
Gold,97,15,0,21,133
Silver,523,446,1,135,1105
Tin,1271,21,518,758,2568
All,2542,1480,520,1006,5548


In [9]:
subset_horeca = tizi_ouzou_combi.loc[tizi_ouzou_combi['Type Canal'].isin(['Horeca'])]
print("dataframe dimensions:", subset_horeca.shape)
subset_horeca['ADM2_EN'].value_counts().reset_index().rename(columns={"index":"Communes","ADM2_EN":"Outlets"}).head(20)

dataframe dimensions: (2473, 96)


,Communes,Outlets
0,Tizi Ouzou,608
1,Draa Ben Khedda,150
2,Freha,125
3,Azazga,98
4,Tigzirt,90
5,Boghni,85
6,Ouadhia,83
7,Bouzguen,67
8,Makouda,66
9,Mekla,64


In [10]:
subset_thresh = form_subset(
  data=subset_horeca, 
  group_var='ADM2_EN', 
  equal_brackets=0, 
  top_field='NSR',
  sample_size=100,
  gold_portion=0.15,
  bronze_portion=0.1
)
 
subset_thresh

Number of items proportionate to group size!


,CUSTOMER_COD,ADM2_EN,Unnamed: 0,Groupe,OrgCm,CDis,SA,Titre,Anc.Client,Nom,...,osm_count,dist_km,NSR,GSR,UNIT_CASES,TRANSACTIONS,NSR_rank,Commune_y,urba_gov_Population,urba_gov_Superficie
0,37832,Ain El Hammam,3113,ZA04,A000,DD,10,Monsieur,1100108.0,CHABA AKLI,...,9,0.598761,2865.209961,2898.154748,757.326700,9380.0,821,Ain El Hammam,20401.0,38.55
1,38867,Azazga,1634,ZA04,A000,DD,10,Monsieur,1101163.0,OUGACHE KAMEL,...,2,0.429676,1257.254905,1275.350501,285.258685,5790.0,1796,Azazga,34683.0,77.05
2,52322,Beni Douala,4867,ZA04,A000,DD,10,Monsieur,NaN,LADJADJ MOURAD,...,0,4.623919,4323.772863,4361.556065,1183.367765,13980.0,455,Beni Douala,21551.0,35.08
3,54407,Bouzguen,4693,ZA04,A000,DD,10,Monsieur,NaN,MOUDJEB AMAR,...,2,0.598761,3131.501270,3163.239883,920.662940,8678.0,724,NaN,NaN,NaN
4,52393,Freha,236,ZA04,A000,DD,10,Monsieur,NaN,MALLEK IDIR,...,0,5.191190,6345.284588,6414.487029,1547.466735,23154.0,233,Freha,24228.0,68.55
5,41201,Freha,1920,ZA04,A000,DD,10,Monsieur,1104091.0,SALEMKOUR BESSAM,...,10,0.429676,4893.812806,4909.646522,1471.318190,14332.0,380,Freha,24228.0,68.55
6,41524,Larba Nait Irathen,3891,ZA04,A000,DD,10,Monsieur,1104425.0,DJEMA HOCINE,...,8,7.235475,8566.522060,8645.668051,2246.748165,31082.0,128,NaN,NaN,NaN
7,38185,Makouda,5176,ZA04,A000,DD,10,Monsieur,1100467.0,BOURAYA HOCINE,...,0,8.024678,2647.730387,2679.973191,707.840590,9596.0,925,Makouda,23388.0,57.43
8,56171,Mekla,3625,ZA04,A000,DD,10,Monsieur,NaN,CHEHRITE RABAH,...,0,8.024678,6383.224049,6445.065314,1103.456980,19396.0,228,Mekla,24237.0,64.71
9,46060,Tigzirt,5052,ZA04,A000,DD,10,Société,NaN,TIZI LUNCH,...,0,3.012929,4572.627916,4634.294336,1217.190120,13360.0,422,Tigzirt,11962.0,41.68


# Form a subset to validate TA matching against

In [11]:
# 1) Build selector
select_number = construct_selector(subset_horeca,
                                   equal_brackets=0,
                                   sample_size=225,
                                   gold_portion=0.2,
                                   bronze_portion=0,
                                   )

select_gold =  round(select_number['n_gold']).astype(int)
select_silver =  round(select_number['n_silver']).astype(int)
select_bronze =  round(select_number['n_bronze']).astype(int)

Number of items proportionate to group size!


In [12]:
# 2 define parameters
TIER="Classification client"
group_var="ADM2_EN"
top_field_short='NSR_rank'

In [13]:
# 3) Form  subset 
# 3.1) gold set
gold=lambda dfg: dfg.nsmallest(select_gold[dfg.name], columns=top_field_short)
gold = subset_horeca.loc[subset_horeca[TIER]=='Gold'].groupby(group_var).apply(gold).reset_index(drop =True)
gold.shape

(22, 96)

In [14]:
# 3.1) bronze set
bronze=lambda dfg: dfg.nsmallest(select_bronze[dfg.name], columns=top_field_short)
bronze= subset_horeca.loc[subset_horeca[TIER]=='Bronze'].groupby(group_var).apply(bronze).reset_index(drop =True)
bronze.shape

(0, 96)

In [15]:
silver=lambda dfg: dfg.nsmallest(select_silver[dfg.name], columns=top_field_short)
silver= subset_horeca.loc[subset_horeca[TIER]=='Silver'].groupby(group_var).apply(silver).reset_index(drop =True)
silver.shape

(176, 96)

In [16]:
subset_thresh =gold.append(silver).append(bronze).sort_values(by=[group_var, TIER])

# Set stats


### Subset Breakdowns


In [17]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_rows = 100
pd.options.display.max_columns= 100

In [18]:
# Tier
pd.crosstab(subset_thresh[TIER], subset_thresh['ADM2_EN'], values=subset_thresh["CUSTOMER_COD"], aggfunc='count', margins=True). \
  sort_values(by=['All'], ascending=False).fillna(0)

ADM2_EN,Abi Youcef,Aghni Goughran,Aghrib,Ain El Hammam,Ain Zaouia,Ait Aissa Mimoun,Ait Bouadou,Ait Khelili,Ait Mahmoud,Ait Toudert,Ait Yahia,Ait Yahia Moussa,Azazga,Azeffoun,Beni Aissi,Beni Douala,Beni Yenni,Beni Zmenzer,Boghni,Boudjima,Bounouh,Bouzguen,Draa Ben Khedda,Draa El Mizan,Freha,Iboudraren,Idjeur,Iferhounene,Ifigha,Iflissen,Iloula Oumalou,Irdjen,Larba Nait Irathen,Maatka,Makouda,Mechtrass,Mekla,Mizrana,Ouacif,Ouadhia,Ouaguenoun,Sidi Naamane,Souamaa,Tadmait,Tigzirt,Timizart,Tirmitine,Tizi N'Thleta,Tizi Ouzou,Tizi Rached,Yakouren,All
Classification client,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,1.00,1.00,1.00,4.00,1.00,2.00,2.00,1.00,1.00,1.00,1.00,1.00,8.00,3.00,2.00,5.00,1.00,1.00,6.00,2.00,1.00,6.00,11.00,4.00,11.00,1.00,1.00,1.00,1.00,2.00,1.00,2.00,5.00,3.00,6.00,1.00,6.00,1.00,2.00,6.00,2.00,2.00,1.00,4.00,7.00,3.00,2.00,1.00,52.00,5.00,2.00,198
Silver,1.00,1.00,1.00,3.00,1.00,2.00,2.00,1.00,1.00,1.00,1.00,1.00,7.00,2.00,2.00,4.00,1.00,1.00,6.00,2.00,1.00,5.00,11.00,4.00,9.00,1.00,1.00,1.00,1.00,2.00,1.00,2.00,4.00,2.00,5.00,1.00,5.00,1.00,2.00,6.00,2.00,2.00,1.00,4.00,6.00,2.00,2.00,1.00,44.00,4.00,2.00,176
Gold,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,8.00,1.00,0.00,22


In [19]:
# Total NSR by tier
pd.crosstab(subset_thresh[TIER], subset_thresh['ADM2_EN'], values=subset_thresh['NSR'], aggfunc='sum', margins=True). \
  sort_values(by=['All'], ascending=False).fillna(0)

ADM2_EN,Abi Youcef,Aghni Goughran,Aghrib,Ain El Hammam,Ain Zaouia,Ait Aissa Mimoun,Ait Bouadou,Ait Khelili,Ait Mahmoud,Ait Toudert,Ait Yahia,Ait Yahia Moussa,Azazga,Azeffoun,Beni Aissi,Beni Douala,Beni Yenni,Beni Zmenzer,Boghni,Boudjima,Bounouh,Bouzguen,Draa Ben Khedda,Draa El Mizan,Freha,Iboudraren,Idjeur,Iferhounene,Ifigha,Iflissen,Iloula Oumalou,Irdjen,Larba Nait Irathen,Maatka,Makouda,Mechtrass,Mekla,Mizrana,Ouacif,Ouadhia,Ouaguenoun,Sidi Naamane,Souamaa,Tadmait,Tigzirt,Timizart,Tirmitine,Tizi N'Thleta,Tizi Ouzou,Tizi Rached,Yakouren,All
Classification client,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,1583.11,470.19,2389.12,11517.47,3417.51,6293.85,2818.95,1458.68,356.49,1188.42,2583.98,95.97,10491.74,2177.99,6357.39,12759.05,847.98,1056.58,11799.90,4826.13,1400.07,21289.39,8714.89,4309.16,38437.19,1060.65,3024.87,1120.05,1696.65,4854.49,859.01,5678.12,16785.32,3524.41,12281.27,1625.05,15148.87,2532.91,4042.41,14469.52,2766.83,1129.71,210.13,4095.52,18894.66,8169.39,4118.50,783.63,112961.84,10033.72,1190.31,411699.06
Silver,1583.11,470.19,2389.12,8652.26,3417.51,6293.85,2818.95,1458.68,356.49,1188.42,2583.98,95.97,9234.49,1939.12,6357.39,8435.28,847.98,1056.58,11799.90,4826.13,1400.07,18157.89,8714.89,4309.16,27198.10,1060.65,3024.87,1120.05,1696.65,4854.49,859.01,5678.12,8218.79,2454.31,9633.54,1625.05,8765.64,2532.91,4042.41,14469.52,2766.83,1129.71,210.13,4095.52,14322.03,4158.42,4118.50,783.63,86746.48,6089.34,1190.31,331232.43
Gold,0.00,0.00,0.00,2865.21,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1257.25,238.87,0.00,4323.77,0.00,0.00,0.00,0.00,0.00,3131.50,0.00,0.00,11239.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8566.52,1070.10,2647.73,0.00,6383.22,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4572.63,4010.97,0.00,0.00,26215.37,3944.38,0.00,80466.63


In [20]:
# Derived urbanicity
pd.crosstab(subset_thresh['urbanicity_overall'], subset_thresh['ADM2_EN'], margins=True). \
  sort_values(by=['All'], ascending=False)

ADM2_EN,Abi Youcef,Aghni Goughran,Aghrib,Ain El Hammam,Ain Zaouia,Ait Aissa Mimoun,Ait Bouadou,Ait Khelili,Ait Mahmoud,Ait Toudert,Ait Yahia,Ait Yahia Moussa,Azazga,Azeffoun,Beni Aissi,Beni Douala,Beni Yenni,Beni Zmenzer,Boghni,Boudjima,Bounouh,Bouzguen,Draa Ben Khedda,Draa El Mizan,Freha,Iboudraren,Idjeur,Iferhounene,Ifigha,Iflissen,Iloula Oumalou,Irdjen,Larba Nait Irathen,Maatka,Makouda,Mechtrass,Mekla,Mizrana,Ouacif,Ouadhia,Ouaguenoun,Sidi Naamane,Souamaa,Tadmait,Tigzirt,Timizart,Tirmitine,Tizi N'Thleta,Tizi Ouzou,Tizi Rached,Yakouren,All
urbanicity_overall,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,1,1,1,4,1,2,2,1,1,1,1,1,8,3,2,5,1,1,6,2,1,6,11,4,11,1,1,1,1,2,1,2,5,3,6,1,6,1,2,6,2,2,1,4,7,3,2,1,52,5,2,198
1,0,0,0,2,0,1,0,0,0,0,1,0,5,0,2,1,0,1,6,0,0,4,10,2,3,1,1,1,0,1,1,2,0,0,2,0,3,0,1,3,1,0,0,2,2,0,0,0,48,5,0,112
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,1,3,0,0,0,1,0,0,0,1,0,1,1,0,0,0,3,0,0,0,0,2,3,1,1,3,0,0,25
3,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,1,0,0,0,0,4,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,13
10,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,2,1,0,0,0,1,0,1,13
4,0,0,0,0,0,0,2,0,1,0,0,0,1,1,0,1,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,10
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,2,0,0,1,0,0,0,0,0,0,9
6,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
8,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5


In [21]:
# Total NSR by derived urbanicity
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.crosstab(subset_thresh['urbanicity_overall'], subset_thresh['ADM2_EN'], values=subset_thresh['NSR'], aggfunc='sum', margins=True). \
  sort_values(by=['All'], ascending=False).fillna(0)

ADM2_EN,Abi Youcef,Aghni Goughran,Aghrib,Ain El Hammam,Ain Zaouia,Ait Aissa Mimoun,Ait Bouadou,Ait Khelili,Ait Mahmoud,Ait Toudert,Ait Yahia,Ait Yahia Moussa,Azazga,Azeffoun,Beni Aissi,Beni Douala,Beni Yenni,Beni Zmenzer,Boghni,Boudjima,Bounouh,Bouzguen,Draa Ben Khedda,Draa El Mizan,Freha,Iboudraren,Idjeur,Iferhounene,Ifigha,Iflissen,Iloula Oumalou,Irdjen,Larba Nait Irathen,Maatka,Makouda,Mechtrass,Mekla,Mizrana,Ouacif,Ouadhia,Ouaguenoun,Sidi Naamane,Souamaa,Tadmait,Tigzirt,Timizart,Tirmitine,Tizi N'Thleta,Tizi Ouzou,Tizi Rached,Yakouren,All
urbanicity_overall,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
All,1583.11,470.19,2389.12,11517.47,3417.51,6293.85,2818.95,1458.68,356.49,1188.42,2583.98,95.97,10491.74,2177.99,6357.39,12759.05,847.98,1056.58,11799.90,4826.13,1400.07,21289.39,8714.89,4309.16,38437.19,1060.65,3024.87,1120.05,1696.65,4854.49,859.01,5678.12,16785.32,3524.41,12281.27,1625.05,15148.87,2532.91,4042.41,14469.52,2766.83,1129.71,210.13,4095.52,18894.66,8169.39,4118.50,783.63,112961.84,10033.72,1190.31,411699.06
1,0.00,0.00,0.00,5839.14,0.00,3354.58,0.00,0.00,0.00,0.00,2583.98,0.00,4845.78,0.00,6357.39,2139.27,0.00,1056.58,11799.90,0.00,0.00,16220.46,7287.00,2567.86,9999.98,1060.65,3024.87,1120.05,0.00,3300.11,859.01,5678.12,0.00,0.00,3810.95,0.00,6265.00,0.00,2060.77,7396.46,647.58,0.00,0.00,2076.18,3883.98,0.00,0.00,0.00,103871.67,10033.72,0.00,229141.02
2,0.00,470.19,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4371.77,0.00,0.00,0.00,0.00,0.00,0.00,1427.90,835.55,9605.70,0.00,0.00,0.00,1696.65,0.00,0.00,0.00,1826.30,0.00,1912.23,1625.05,0.00,0.00,0.00,7073.06,0.00,0.00,0.00,0.00,7113.92,8169.39,2751.55,783.63,6365.04,0.00,0.00,56027.94
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4235.73,1199.78,0.00,0.00,0.00,0.00,0.00,2117.12,0.00,0.00,0.00,0.00,15303.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2304.47,0.00,1362.01,0.00,1414.91,2532.91,0.00,0.00,2119.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,32589.68
10,0.00,0.00,2389.12,0.00,3417.51,2939.27,0.00,0.00,0.00,1188.42,0.00,0.00,0.00,0.00,0.00,0.00,847.98,0.00,0.00,2709.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,419.38,0.00,0.00,0.00,0.00,1981.65,0.00,0.00,0.00,0.00,2019.34,2931.49,0.00,0.00,0.00,2725.13,0.00,397.52,23965.83
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1458.68,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3528.00,0.00,0.00,0.00,0.00,1554.39,0.00,0.00,0.00,1070.10,2647.73,0.00,6383.22,0.00,0.00,0.00,0.00,1129.71,0.00,0.00,2155.42,0.00,0.00,0.00,0.00,0.00,0.00,19927.25
4,0.00,0.00,0.00,0.00,0.00,0.00,2818.95,0.00,356.49,0.00,0.00,0.00,1410.23,238.87,0.00,4323.77,0.00,0.00,0.00,0.00,0.00,5068.93,0.00,905.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1366.94,0.00,0.00,0.00,0.00,16489.95
6,1583.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,95.97,0.00,739.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11491.19,2034.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15944.54
8,0.00,0.00,0.00,5678.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2548.36,0.00,1085.73,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2809.84,0.00,0.00,0.00,0.00,0.00,0.00,12122.25


In [22]:
# Gov metrics by derived urbanicity
pd.crosstab(subset_thresh['urbanicity_overall'], subset_thresh['ADM2_EN'], values=subset_thresh['urba_gov_Population'], aggfunc='sum', margins=True).fillna(0)

ADM2_EN,Abi Youcef,Aghni Goughran,Aghrib,Ain El Hammam,Ain Zaouia,Ait Aissa Mimoun,Ait Bouadou,Ait Khelili,Ait Mahmoud,Ait Toudert,Ait Yahia,Ait Yahia Moussa,Azazga,Azeffoun,Beni Aissi,Beni Douala,Beni Yenni,Beni Zmenzer,Boghni,Boudjima,Bounouh,Bouzguen,Draa Ben Khedda,Draa El Mizan,Freha,Iboudraren,Idjeur,Iferhounene,Ifigha,Iflissen,Iloula Oumalou,Irdjen,Larba Nait Irathen,Maatka,Makouda,Mechtrass,Mekla,Mizrana,Ouacif,Ouadhia,Ouaguenoun,Sidi Naamane,Souamaa,Tadmait,Tigzirt,Timizart,Tirmitine,Tizi N'Thleta,Tizi Ouzou,Tizi Rached,Yakouren,All
urbanicity_overall,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.00,0.00,0.00,40802.00,0.00,20268.00,0.00,0.00,0.00,0.00,14439.00,0.00,173415.00,0.00,15256.00,21551.00,0.00,0.00,187578.00,0.00,0.00,0.00,313820.00,77772.00,72684.00,5398.00,10301.00,0.00,0.00,14311.00,0.00,26298.00,0.00,0.00,46776.00,0.00,72711.00,0.00,0.00,47313.00,17425.00,0.00,0.00,45676.00,23924.00,0.00,0.00,0.00,6484224.00,85805.00,0.00,7817747.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,43102.00,0.00,0.00,0.00,0.00,0.00,0.00,31382.00,38886.00,72684.00,0.00,0.00,0.00,9160.00,0.00,0.00,0.00,0.00,0.00,23388.00,0.00,0.00,0.00,0.00,47313.00,0.00,0.00,0.00,0.00,23924.00,86988.00,19027.00,0.00,405264.00,0.00,0.00,801118.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,69366.00,16847.00,0.00,0.00,0.00,0.00,0.00,15628.00,0.00,0.00,0.00,0.00,96912.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23388.00,0.00,24237.00,9469.00,0.00,0.00,17425.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,273272.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7699.00,0.00,0.00,0.00,34683.00,16847.00,0.00,21551.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38886.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19027.00,0.00,0.00,0.00,0.00,138693.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21551.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12203.00,33754.00
6,7693.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20426.00,0.00,16847.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,44966.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24228.00,0.00,0.00,0.00,0.00,14311.00,0.00,0.00,0.00,0.00,23388.00,0.00,24237.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11962.00,0.00,0.00,0.00,0.00,0.00,0.00,98126.00
8,0.00,0.00,0.00,40802.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,23388.00,0.00,24237.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11962.00,0.00,0.00,0.00,0.00,0.00,0.00,100389.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9731.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9954.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19685.00


# Export DATA

In [23]:
subset_thresh.to_csv("horeca_tz_customer_subset.csv")

In [24]:
subset_thresh.shape

(198, 96)